<a href="https://colab.research.google.com/github/Yogananth-r/face-recognition/blob/main/misc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

In [ ]:
import cv2
import os
import numpy as np
# Load the cascade classifier for face detection
face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Load the photos in the folder
def get_images_and_labels(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        # Load the image
        #print("hi ",filename)
        image = cv2.imread(os.path.join(folder, filename))
        if image is None:
          print(f"Failed to load image: {filename}")
          continue


        # Convert the image to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Detect faces in the image
        faces = face_detector.detectMultiScale(gray, 1.3, 5)

        # For each face, extract a sample and add it to the training data
        for (x, y, w, h) in faces:
            images.append(gray[y:y + h, x:x + w])
            label_string = filename.split("_")[1].split(".")[0]
            #print("ls",label_string)
            if label_string:
              labels.append(label_string)

    return images, labels


# Train the face recognition model
def train_model(folder):
    images, labels = get_images_and_labels(folder)

    # Train the model using the Local Binary Patterns Histograms (LBPH) algorithm
    model = cv2.face.LBPHFaceRecognizer_create()
    labels = np.array(labels, dtype=np.int32)
    model.train(images, np.array(labels))

    return model

# Test the face recognition model
def test_model(model, folder):
    images, labels = get_images_and_labels(folder)
    print(labels)
    # Predict the labels for each image
    predicted_labels = []
    for image in images:
        label, confidence = model.predict(image)
        predicted_labels.append(label)

    # Calculate the accuracy of the model
    accuracy = np.count_nonzero(np.array(predicted_labels) == np.array(labels)) / len(labels)
    return accuracy

# Train and test the model
model = train_model("/content/sample_data/data")
accuracy = test_model(model, "/content/sample_data/data")
print("Accuracy: {:.2f}%".format(accuracy * 100))

Failed to load image: .ipynb_checkpoints
Failed to load image: .ipynb_checkpoints
['11', '7', '5', '5', '10', '12', '1', '2', '3', '3', '8', '13', '4', '6', '6']
Accuracy: 0.00%


<ipython-input-30-07f0fb55021d>:59: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  accuracy = np.count_nonzero(np.array(predicted_labels) == np.array(labels)) / len(labels)


In [ ]:
def use_camera():
  js = Javascript('''
    async function use_camera() {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', 0.8);
    }
    ''')
  display(js)
  data = eval_js('use_camera()')
  binary = b64decode(data.split(',')[1])
  with open('photo.jpg', 'wb') as f:
    f.write(binary)
  return len(binary)

result = use_camera()
print(result)

<IPython.core.display.Javascript object>

44010


In [ ]:
def test(image):
  face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
  image = cv2.imread(image)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


  faces = face_detector.detectMultiScale(gray, 1.3, 5)
  for (x, y, w, h) in faces:
    face = gray[y:y + h, x:x + w]
    label, confidence = model.predict(face)
    """if label in range(1,9):
      label="yogi"
    else:
      label="raju" """
    print("Label: {} Confidence: {:.2f}%".format(label, confidence))

In [ ]:
test("/content/photo.jpg")

Label: 5 Confidence: 69.61%
